In [6]:
from rosbags.highlevel import AnyReader
import pandas as pd
from pathlib import Path

In [8]:
bag_path = Path('dist_bearing_bag')
topics = [
    '/bearing_dist/bearing',
    '/bearing_dist/dist',
    '/bearing_dist/bearing_error',
    '/bearing_dist/dist_error'
]

# Create a reader
with AnyReader([bag_path]) as reader:
    # Inspect available topics
    print("Available topics:")
    for connection in reader.connections:
        print(f"  {connection.topic} ({connection.msgtype})")

    # Prepare lists for data
    records = {topic: {'time': [], 'value': []} for topic in topics}

    # Read messages
    for connection, timestamp, rawdata in reader.messages():
        if connection.topic in topics:
            msg = reader.deserialize(rawdata, connection.msgtype)
            # Assuming messages are std_msgs/Float64
            records[connection.topic]['time'].append(timestamp * 1e-9)  # sec
            records[connection.topic]['value'].append(msg.data)

# Build a pandas DataFrame
dfs = []
for topic in topics:
    df = pd.DataFrame({
        'time': records[topic]['time'],
        topic: records[topic]['value']
    })
    dfs.append(df)

# Merge all dataframes on nearest timestamp
df_merged = dfs[0]
for df in dfs[1:]:
    df_merged = pd.merge_asof(df_merged.sort_values('time'),
                              df.sort_values('time'),
                              on='time',
                              direction='nearest')

print(df_merged.head())


AnyReaderError: Some database files are missing: ['dist_bearing_bag/rosbag2_2025_11_07-16_27_47_0.db3']